In [2]:
import pandas as pd
from glob import glob
import os.path as osp
import os
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
import json
from scipy import sparse
import shutil
import scipy.io as io

In [4]:
fp = '/datasets/dsc180a-wi20-public/Malware/group_data/group_02/sensitive_data'
COMM_DIR = osp.join('raw', 'comments', '*.csv')
LABL_DIR = osp.join('interim', 'label', '*.csv')
POST_DIR = osp.join('raw', 'posts', '*.csv')
OUT_DIR = osp.join('interim', 'graph')

In [6]:
comm = osp.join(fp, COMM_DIR)
post = osp.join(fp, POST_DIR)
labl = osp.join(fp, LABL_DIR)
comm = pd.concat([pd.read_csv(i, usecols = ['id', 'author', 'parent_id', 'link_id']
                    ) for i in glob(comm)])
post = pd.concat([pd.read_csv(i, usecols = ['id', 'author', 'subreddit']
                    ) for i in glob(post)])
labl = pd.concat([pd.read_csv(i) for i in glob(labl)])
labl = labl[labl.label != -1]
post.author = post.author.str.lower()
post.subreddit = post.subreddit.str.lower()
post = post[(post.author != '[deleted]')&(post.author != 'automoderator')& (post.author != 'snapshillbot')]
comm['parent_id'] = comm.parent_id.str[3:]
comm['link_id'] = comm.link_id.str[3:]
comm = comm[['id','author', 'parent_id', 'link_id']]
comm.author = comm.author.str.lower()
comm = comm[(comm.author != '[deleted]')&(comm.author != 'automoderator') & (comm.author != 'snapshillbot')]
comm = comm.dropna()
post = post[(post.id.isin(labl.post_id)) & (post.id.isin(comm.link_id))]
comm = comm[comm.link_id.isin(post.id)]
comm = comm[(comm.parent_id.isin(post.id)) | (comm.parent_id.isin(comm.id)) | (comm.link_id.isin(post.id))]
comm_root = comm[comm.parent_id.str.len() == 6]
comm_nest = comm[comm.parent_id.str.len() == 7]

In [7]:
post_pauthor = post[['id', 'author']]
post_pauthor.columns = ['who', 'whom']
pauthor_post = post[['id', 'author']]
pauthor_post.columns = ['whom', 'who']
pauthor_post_edges = pd.concat([post_pauthor, pauthor_post], ignore_index = True)
cauthor_pauthor_edges = pd.merge(comm_root[['author', 'parent_id']], post[['id', 'author']], \
        left_on = 'parent_id', right_on = 'id', how = 'left')[['author_x', 'author_y']]
cauthor_pauthor_edges.columns = ['who', 'whom']
cauthor_cauthor = pd.merge(comm_nest[['parent_id', 'author']], comm_nest[['id', 'author']], \
        left_on = 'parent_id', right_on = 'id', how = 'left')[['author_x', 'author_y']]
cauthor_cauthor.columns = ['whom', 'who']
edges = pd.concat([pauthor_post_edges, cauthor_pauthor_edges, cauthor_cauthor], ignore_index = True)

In [9]:
post_names = post.id.unique()
node_names = np.unique(np.append(edges.who.dropna().values, edges.whom.dropna().values))
node_maps = pd.DataFrame(
            {
                'id': np.arange(len(node_names)),
                'name': node_names
            }
)

In [11]:
edge_pairs = edges.dropna().who + edges.dropna().whom
edge_pair = edges.copy()
pair_counts = edge_pairs.value_counts()
edge_pair['pairs'] = edge_pairs
pair_counts = pair_counts.astype(int).to_frame()

In [12]:
edges = pd.merge(edge_pair, pair_counts, left_on = 'pairs', right_index = True)[['who', 'whom', 0]]
edge_idx = edges.drop_duplicates()
edge_idx = pd.merge(edge_idx, node_maps, left_on='who', right_on='name', how='left')[['id', 'whom', 0]]
edge_idx.columns = ['who_id', 'whom', 'weight']
edge_idx = pd.merge(edge_idx, node_maps, left_on='whom', right_on='name', how='left')[['who_id', 'id', 'weight']]
edge_idx.columns = ['who_id', 'whom_id', 'weight']
edge_weight = edge_idx[['weight']].values
edge_idx = edge_idx.sort_values(['who_id', 'whom_id'])
edge_idx_ = edge_idx[['who_id', 'whom_id']].values

In [16]:
N = sparse.csr_matrix((edge_weight.reshape(-1,), (edge_idx_[:, 0], edge_idx_[:, 1])), \
                                shape = (node_maps.shape[0], node_maps.shape[0]))

In [33]:
post_mask = node_maps.name.isin(post_names)
post_indx = node_maps[post_mask].id.values

In [35]:
post_indx.shape

(103413,)

In [34]:
from scipy import io
import torch
from torch_geometric.utils import from_scipy_sparse_matrix


In [28]:
g = io.loadmat('/datasets/dsc180a-wi20-public/Malware/group_data/group_02/sensitive_data/interim/graph/graph_2.mat')

In [29]:
N = t['N']

In [30]:
p_cate = g['post_cate'].toarray()

In [33]:
p_cate

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [35]:
edge_idx, x =from_scipy_sparse_matrix(N)

In [43]:
edge_idx

tensor([[577328,  45380,  82809,  ..., 263087, 298545, 421156],
        [     0,      1,      1,  ..., 585317, 585317, 585319]])